In [ ]:
!pip install crewai
!pip install 'crewai[tools]'
!pip install langchain_groq

  Using cached crewai_tools-0.4.7-py3-none-any.whl (65 kB)
  Using cached docker-7.1.0-py3-none-any.whl (147 kB)
  Using cached docx2txt-0.8.tar.gz (2.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached lancedb-0.5.7-py3-none-any.whl (115 kB)
  Using cached pyright-1.1.370-py3-none-any.whl (18 kB)
  Using cached pytest-8.2.2-py3-none-any.whl (339 kB)
  Using cached pytube-15.0.0-py3-none-any.whl (57 kB)
  Using cached selenium-4.22.0-py3-none-any.whl (9.4 MB)
  Using cached deprecation-2.1.0-py2.py3-none-any.whl (11 kB)
  Using cached pylance-0.9.18-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (21.6 MB)
  Using cached ratelimiter-1.2.0.post0-py3-none-any.whl (6.6 kB)
  Using cached retry-0.9.2-py2.py3-none-any.whl (8.0 kB)
  Using cached semver-3.0.2-py3-none-any.whl (17 kB)
  Using cached nodeenv-1.9.1-py2.py3-none-any.whl (22 kB)
  Using cached trio-0.26.0-py3-none-any.whl (475 kB)
  Using cached trio_websocket-0.11.1-py3-none-any.whl (17 kB)
     ━━━━━━━━━━━━━━

In [ ]:
import os
from crewai import Agent,Task,Crew,Process
from crewai_tools import SerperDevTool
from langchain_groq import ChatGroq

# Set up environment variable for Grok API key
os.environ['GROQ_API_KEY'] = "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"

In [ ]:
# Initialize LLM with Grok API key
llm = ChatGroq(model="gemma-7b-it",groq_api_key=os.environ['GROQ_API_KEY'])
print(llm.invoke('hi'))

content="Hi! 👋 It's great to hear from you. How can I help you today? 😊" response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 15, 'total_tokens': 36, 'completion_time': 0.024106215, 'prompt_time': 0.009649956, 'queue_time': None, 'total_time': 0.033756171}, 'model_name': 'gemma-7b-it', 'system_fingerprint': 'fp_e3b292b95f', 'finish_reason': 'stop', 'logprobs': None} id='run-6ee1dee0-3100-4c03-b012-bed2d72bfa51-0' usage_metadata={'input_tokens': 15, 'output_tokens': 21, 'total_tokens': 36}


In [ ]:
# Define the assignment text and submission

assignment_text = """
  Assignment: Automated FAQ Answering

  Goal:

  Create a prompt that answers Frequently Asked Questions (FAQs) for a product of your choice.

  First, have the model generate examples of FAQs and corresponding answers based on a brief description of the product (maybe choose an item that you use in your everyday life). Then, use these examples as context for the model to learn from when answering a novel question about the product (you can write this novel question yourself).
"""

assignmentsubmission = """
Frequently Asked Questions (FAQs)

1. What is the processor type in the V15 G2-ITL Laptop (Lenovo)?
The V15 G2-ITL Laptop is equipped with an 11th Generation Intel® Core™ i3-1115G4 Processor.

2. How much memory does the laptop have?
The laptop comes with 8GB of DDR4 2666 memory.

3. What operating system is pre-installed on the laptop?
The laptop runs Windows 11 Home Single Language 64-bit (English).

4. What storage options are available in this laptop?
The laptop includes a 256GB PCIe NVMe SSD and a 1TB HDD (5400rpm).

5. Does the laptop have wireless connectivity?
Yes, it supports Wireless 802.11 2x2 AC and Bluetooth® 5.1 or above.

6. What ports are available on the V15 G2-ITL Laptop?
The laptop includes the following ports:
- 1x SD Card reader
- HDMI
- 1x USB 2.0
- 1x USB 3.2 Gen1 Type C
- 1x USB Type C
- DC Jack (round)
- Combo of 3.5mm Stereo Headphone Output

7. What kind of camera does the laptop have?
The laptop has a 1MP camera with an array microphone and a privacy shutter.

8. What graphics does the laptop feature?
The laptop is equipped with Intel UHD Graphics.

9. What is the size and resolution of the laptop’s monitor?
The laptop has a 15.6" Full HD (FHD) monitor.

10. What kind of warranty is included with the laptop?
The laptop comes with a Courier Carry-in warranty.

Novel Question : Does the Lenovo - V15 G2-ITL Laptop have USB Type C?

ChatGPT: Yes, the Lenovo V15 G2-ITL Laptop has two USB Type C ports. One is a USB 3.2 Gen1 Type C port, and the other is a standard USB Type C port.

Novel Question 1 : Does the Laptop have Windows 11 or Windows 10?
Novel Question 2 : What is the Total Storage of the Laptop?

ChatGPT:  Novel Question 1: Does the Laptop have Windows 11 or Windows 10?
          - The Lenovo V15 G2-ITL Laptop comes pre-installed with Windows 11 Home Single Language 64-bit.

          Novel Question 2: What is the Total Storage of the Laptop?
          - The Lenovo V15 G2-ITL Laptop has a total storage capacity of 1.256TB, consisting of a 256GB PCIe NVMe SSD and a 1TB HDD (5400rpm).
"""

In [ ]:
# Define agents with roles and goals

clarity_and_completeness_agent = Agent(
    role='Clarity and Completeness Grader',
    goal='Evaluate the clarity and completeness of the assignment submission',
    backstory=f"""You are an experienced educator with a keen eye for detail, specializing in evaluating the clarity and completeness of written content.
    Assignment: {assignment_text}""",
    verbose=True,
    llm=llm,
    allow_delegation=False
)

relevance_of_product_agent = Agent(
    role='Relevance and Focus Grader',
    goal='Evaluate the relevance and focus of the assignment submission',
    backstory=f"""You are a seasoned educator with a strong background in assessing the relevance and focus of academic work.
    Assignment: {assignment_text}""",
    verbose=True,
    llm=llm,
    allow_delegation=False
)

accuracy_of_answer_agent = Agent(
    role='Accuracy Grader',
    goal='Evaluate the accuracy of the information in the assignment submission',
    backstory=f"""You are an expert in evaluating the accuracy of content, ensuring that all facts and details are correctly represented.
    Assignment: {assignment_text}""",
    verbose=True,
    llm=llm,
    allow_delegation=False
)

context_agent = Agent(
    role='Context Grader',
    goal='Evaluate the context of the assignment submission',
    backstory=f"""You are a seasoned educator with a strong background in assessing the context of academic work.
    Assignment: {assignment_text}""",
    verbose=True,
    llm=llm,
    allow_delegation=False
)

novel_question_agent = Agent(
    role='Novel Question Grader',
    goal='Evaluate the relevance and focus of the novel question',
    backstory=f"""You are a seasoned educator with a strong background in assessing the relevance and focus of academic work.
    Assignment: {assignment_text}""",
    verbose=True,
    llm=llm,
    allow_delegation=False
)

task_clarity_and_completeness = Task(
    description=f"""Grade the clarity and completeness of the following assignment submission on a scale of 1-20:
    {assignmentsubmission}""",
    expected_output="Score for clarity and conciseness with comments",
    agent=clarity_and_completeness_agent
)

task_relevance = Task(
    description=f"""Grade the relevance and focus of the following assignment submission on a scale of 1-30:
    {assignmentsubmission}""",
    expected_output="Score for relevance and focus with comments",
    agent=relevance_of_product_agent
)

task_accuracy = Task(
    description=f"""Grade the accuracy of the information in the following assignment submission on a scale of 1-20:
    {assignmentsubmission}""",
    expected_output="Score for accuracy with comments",
    agent=accuracy_of_answer_agent
)

task_context = Task(
    description=f"""Grade the context of the following assignment submission on a scale of 1-20:
    {assignmentsubmission}""",
    expected_output="Score for context with comments",
    agent=context_agent
)

task_novel_question = Task(
    description=f"""Grade the relevance and focus of the following novel question
    {assignment_text}""",
    expected_output="Score for relevance and focus with comments",
    agent=novel_question_agent
)

# Initialize and execute the crew
crew = Crew(
    agents=[clarity_and_completeness_agent, relevance_of_product_agent, accuracy_of_answer_agent, context_agent, novel_question_agent],
    tasks=[task_clarity_and_completeness, task_relevance, task_accuracy, task_context, task_novel_question],
    verbose=2,
    process=Process.sequential
)

# Start the grading process
result = crew.kickoff()

# Print the result
print("######################")
print(result)

 [2024-07-07 12:00:51][DEBUG]: == Working Agent: Clarity and Completeness Grader
 [2024-07-07 12:00:51][INFO]: == Starting Task: Grade the clarity and completeness of the following assignment submission on a scale of 1-20:
    
Frequently Asked Questions (FAQs)

1. What is the processor type in the V15 G2-ITL Laptop (Lenovo)?
The V15 G2-ITL Laptop is equipped with an 11th Generation Intel® Core™ i3-1115G4 Processor.

2. How much memory does the laptop have?
The laptop comes with 8GB of DDR4 2666 memory.

3. What operating system is pre-installed on the laptop?
The laptop runs Windows 11 Home Single Language 64-bit (English).

4. What storage options are available in this laptop?
The laptop includes a 256GB PCIe NVMe SSD and a 1TB HDD (5400rpm).

5. Does the laptop have wireless connectivity?
Yes, it supports Wireless 802.11 2x2 AC and Bluetooth® 5.1 or above.

6. What ports are available on the V15 G2-ITL Laptop?
The laptop includes the following ports:
- 1x SD Card reader
- HDMI
- 1x 